In [23]:
import numpy as np 
import pandas as pd 
import os
from sklearn.utils import shuffle

##################### New Packages
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables import Keypoint, KeypointsOnImage
from skimage import img_as_ubyte
#####################

from sklearn.model_selection import train_test_split 
from matplotlib import pyplot as plt
%matplotlib inline 

for dirname, _, filenames in os.walk('drive/My Drive/facial_keypoints_data/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

drive/My Drive/facial_keypoints_data/IdLookupTable.csv
drive/My Drive/facial_keypoints_data/SampleSubmission.csv
drive/My Drive/facial_keypoints_data/test/test.csv
drive/My Drive/facial_keypoints_data/training/training.csv


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
training = pd.read_csv('drive/My Drive/facial_keypoints_data/training/training.csv')

In [0]:
training = training.dropna()
# Sample 500 observations from the whole training set
df_imgs = training.sample(500).copy()
df_imgs['Image'] = df_imgs['Image'].apply(lambda x: np.fromstring(x, dtype=int, sep=' ').reshape((96,96)))

In [0]:
# Create an array object of images
X = df_imgs['Image'].values

# Copied it for testing - work can be done on X instead
XX = X.copy()

y = df_imgs[df_imgs.columns[:-1]].values
y = y.astype(np.float32)

In [0]:
# 3-for loops to change every possible object type to uint8 (first array, second array, and the values of [0-255] themselves)
for i, t in enumerate(X):
  for j, tt in enumerate(X[i]):
    for k, ttt in enumerate(X[i][j]):
      XX[i][j][k] = X[i][j][k].astype(np.uint8)
    XX[i][j] = img_as_ubyte(X[i][j])
  XX[i] = img_as_ubyte(X[i])
# Warnings do not matter - don't worry

In [0]:
'''
For loop which takes images and matching keypoints' coordinates, 
makes KeyPoint object out of all of them then it applies same "seq" augmentation
on both the image and keypoints. Finally it unpacks keypoints to cooridnates and
appends augmented image with keypoints to corresponding lists ***_aug.
'''
ia.seed(1)

imgs_aug = []
kpss_aug = []

for j, (image, keypt) in enumerate(zip(XX,y)):
  img = XX[j]
  kps = KeypointsOnImage([
    Keypoint(keypt[0], keypt[1]),
    Keypoint(keypt[2], keypt[3]),
    Keypoint(keypt[4], keypt[5]),
    Keypoint(keypt[6], keypt[7]),
    Keypoint(keypt[8], keypt[9]),
    Keypoint(keypt[10], keypt[11]),
    Keypoint(keypt[12], keypt[13]),
    Keypoint(keypt[14], keypt[15]),
    Keypoint(keypt[16], keypt[17]),
    Keypoint(keypt[18], keypt[19]),
    Keypoint(keypt[20], keypt[21]),
    Keypoint(keypt[22], keypt[23]),
    Keypoint(keypt[24], keypt[25]),
    Keypoint(keypt[26], keypt[27]),
    Keypoint(keypt[28], keypt[29])],
    shape = (96,96))


  seq = iaa.Sequential([
      iaa.Fliplr(1.0)
  ])
  img_aug, kps_aug = seq(image=img, keypoints=kps)
  act_kps = []
  act_img = []
  for i in range(len(kps.keypoints)):
    act_kps.append(kps_aug.keypoints[i].x)
    act_kps.append(kps_aug.keypoints[i].y)
  imgs_aug.append(img_aug)
  kpss_aug.append(act_kps)


In [0]:
# Create arrays from lists

img_arr = np.array(imgs_aug)
key_arr = np.array(kpss_aug)

# Create X_aug and y_aug which are same dimensions as training X and y

X_aug = np.reshape(img_arr, (img_arr.shape[0], -1)) / 255.
X_aug = X_aug.astype(np.float32)

y_aug = (key_arr - 48) / 48
X_aug, y_aug = shuffle(X_aug, y_aug, random_state=42)  # shuffle train data
y_aug = y_aug.astype(np.float32)